# 📚 YouTube Trending Analysis — Steps 1–15


**Author:** Hussein Shamas  
**What:** Clean, reproduce and analyze the YouTube trending dataset. This notebook implements exactly tasks 1→15.

**Problem:** We faced a problem where the CSV files are not properly decoded as UTF-8 and we were reading them with the default encoding (utf-8) but the file was saved in ISO-8859-1 (Latin-1) youTube video titles can be in any language, including Chinese, Russian, Japanese, emojis, etc

---


## 🔧 Setup — imports and display options

In [20]:
# import standard libraries
import pandas as pd  # dataframes
import numpy as np   # numeric helpers
import glob          # file pattern matching
import json          # read json files

# pandas display settings for nicer output
pd.set_option('display.max_columns', None)  # show all columns when printing


## 1) Create a single dataframe with concatenation of all CSV files, adding a `country` column

In [43]:
import os

csv_files = glob.glob('data/csv/*videos.csv')
dfs = []

for file in csv_files:
    # Extract filename safely
    filename = os.path.basename(file)  # e.g., 'CAvideos.csv'
    country_code = filename[:2]        # first 2 chars, 'CA', 'US', etc.
    
    # Try reading with UTF-8, fallback to latin1
    try:
        df = pd.read_csv(file, encoding='utf-8', low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding='latin1', low_memory=False)
    
    # Add country column
    df['country'] = country_code
    
    # Clean text columns
    for col in ['title', 'channel_title', 'tags']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    
    dfs.append(df)

# Concatenate all
if dfs:
    youtube = pd.concat(dfs, ignore_index=True)
    print('Combined shape:', youtube.shape)
    print(youtube['country'].value_counts())  # Should show all 10 countries correctly
else:
    print("No CSV files were successfully read.")

youtube.head()



Combined shape: (375942, 17)
country
US    40949
CA    40881
DE    40840
RU    40739
FR    40724
MX    40451
GB    38916
IN    37352
KR    34567
JP    20523
Name: count, dtype: int64


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA


## 2) Extract all videos that have no tag.

In [44]:
# identify rows where tags is exactly the string '[none]' (common in this dataset)
no_tag_videos = youtube[youtube['tags'] == '[none]']  # filter rows with no tags

# print count and preview
print('Videos without tags:', len(no_tag_videos))
no_tag_videos.head(5)


Videos without tags: 37698


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
41,JwboxqDylgg,17.14.11,Canada Soccer's Women's National Team v USA In...,Canada Soccer,17,2017-11-13T05:53:49.000Z,[none],36311,277,28,13,https://i.ytimg.com/vi/JwboxqDylgg/default.jpg,False,False,False,Canada Soccer's Women's National Team face riv...,CA
58,9B-q8h31Bpk,17.14.11,John Oliver Tackles Louis C.K. And Donald Trum...,TV Shows,22,2017-11-13T04:49:26.000Z,[none],106029,1270,101,181,https://i.ytimg.com/vi/9B-q8h31Bpk/default.jpg,False,False,False,"John Oliver on News, Politics ...",CA
78,1UE5Dq1rvUA,17.14.11,Taylor Swift Perform Ready For It - SNL,Ken Reactz,24,2017-11-12T05:18:02.000Z,[none],320964,8069,285,717,https://i.ytimg.com/vi/1UE5Dq1rvUA/default.jpg,False,False,False,Thanks for watching please subscribe and subsc...,CA
86,pmJQ4KwliX4,17.14.11,"LATEST Q POSTS: ROTHSCHILDS, HOUSE OF SAUD, lL...",James Munder,2,2017-11-12T21:25:40.000Z,[none],116820,1503,139,1066,https://i.ytimg.com/vi/pmJQ4KwliX4/default.jpg,False,False,False,https://pastebin.ca/3930472\n\nSupport My Chan...,CA
98,lHcXhBojpeQ,17.14.11,三屆TVB視帝，拋棄10年青梅竹馬髮妻，為娶小三還不惜與母絕交！,明星百曉生,22,2017-11-12T12:49:50.000Z,[none],88061,47,58,17,https://i.ytimg.com/vi/lHcXhBojpeQ/default.jpg,False,False,False,NaN,CA


## 3) For each channel, determine the total number of views

**Note:** we compute this using the *latest snapshot per video+country* to avoid double-counting the same video's views on multiple trending days.

In [46]:
# Parse trending_date into proper datetime to identify latest snapshots
# Kaggle trending_date format is 'yy.dd.mm' (e.g. '17.14.11' -> 2017-11-14)
youtube['trending_date_parsed'] = pd.to_datetime(youtube['trending_date'], format='%y.%d.%m', errors='coerce')

# create 'latest_snapshot' by taking the last row per (video_id, country) ordered by trending_date_parsed
youtube_sorted = youtube.sort_values(['video_id', 'country', 'trending_date_parsed'])  # sort values so last is latest
latest_snapshot = youtube_sorted.groupby(['video_id', 'country'], as_index=False).last()  # keep last metrics per video+country

# Now compute total views per channel using the latest snapshot to avoid duplicates across days
channel_views = latest_snapshot.groupby('channel_title', as_index=False)['views'].sum().sort_values('views', ascending=False)
channel_views.head(10)

#(If you just sum views from all rows, you get:
#1000 + 1200 + 1500 = 3700 views.
#But the video really has only 1500 views at the end, not 3700) thats why we compute the latest snapshot. 



,channel_title,views
28369,ibighit,1590170234
15523,Marvel Entertainment,1480915768
26826,YouTube Spotlight,1070604343
23074,T-Series,1064917229
6758,Dude Perfect,1062968677
19003,PewDiePie,1011011576
7052,Ed Sheeran,955853396
4567,ChildishGambinoVEVO,870769400
23721,TaylorSwiftVEVO,858453622
22379,Sony Pictures Entertainment,809936786


## 4) Save rows with disabled comments/ratings or error into `excluded`, and remove them from the original dataframe

In [47]:
# create excluded DataFrame with any of the problematic flags set
excluded = youtube[
    (youtube['comments_disabled'] == True) |
    (youtube['ratings_disabled'] == True) |
    (youtube['video_error_or_removed'] == True)
].copy()  # copy to avoid SettingWithCopy warnings

# show how many excluded
print('Excluded shape:', excluded.shape)

# remove those rows from youtube (operate on the full time-series youtube)
youtube = youtube.drop(excluded.index).reset_index(drop=True)
print('Remaining after removal:', youtube.shape)


Excluded shape: (13657, 19)
Remaining after removal: (362285, 19)


## 5) Add a `like_ratio` column storing the ratio between likes and dislikes

We handle divide-by-zero safely by setting ratio to NaN when dislikes is zero or missing.

In [48]:
# convert likes/dislikes to numeric (in case they were strings)
youtube['likes'] = pd.to_numeric(youtube['likes'], errors='coerce')
youtube['dislikes'] = pd.to_numeric(youtube['dislikes'], errors='coerce')

# compute ratio likes/dislikes safely
youtube['like_ratio'] = youtube['likes'] / youtube['dislikes'].replace(0, np.nan)  # replace 0 with NaN to avoid Inf

# show sample
youtube[['likes','dislikes','like_ratio']].head(5)


,likes,dislikes,like_ratio
0,787425,43420,18.135076
1,127794,1688,75.707346
2,146035,5339,27.352500
3,132239,1989,66.485168
4,1634130,21082,77.513044


## 6) Cluster the publish time into 10-minute intervals

In [49]:
# parse publish_time into timezone-aware datetime
youtube['publish_time_parsed'] = pd.to_datetime(youtube['publish_time'], errors='coerce', utc=True)  # parse publish_time

# floor to nearest 10 minutes to create time_interval
youtube['time_interval'] = youtube['publish_time_parsed'].dt.floor('10min')  # group times into 10-minute bins

youtube[['publish_time','publish_time_parsed','time_interval']].head(5)


,publish_time,publish_time_parsed,time_interval
0,2017-11-10T17:00:03.000Z,2017-11-10 17:00:03+00:00,2017-11-10 17:00:00+00:00
1,2017-11-13T17:00:00.000Z,2017-11-13 17:00:00+00:00,2017-11-13 17:00:00+00:00
2,2017-11-12T19:05:24.000Z,2017-11-12 19:05:24+00:00,2017-11-12 19:00:00+00:00
3,2017-11-12T18:01:41.000Z,2017-11-12 18:01:41+00:00,2017-11-12 18:00:00+00:00
4,2017-11-09T11:04:14.000Z,2017-11-09 11:04:14+00:00,2017-11-09 11:00:00+00:00


## 7) For each interval: number of videos, average likes/dislikes

In [50]:
# group by the 10-minute time_interval and compute counts and averages
interval_stats = youtube.groupby('time_interval').agg(
    videos_count=('video_id','count'),  # how many rows/videos in that time interval
    avg_likes=('likes','mean'),         # average likes in that interval
    avg_dislikes=('dislikes','mean')    # average dislikes in that interval
).reset_index()

interval_stats.head(8)


,time_interval,videos_count,avg_likes,avg_dislikes
0,2006-07-23 08:20:00+00:00,1,459.000000,152.000000
1,2007-03-05 16:20:00+00:00,9,336.666667,2.000000
2,2007-06-25 06:50:00+00:00,12,579.833333,11.500000
3,2007-12-03 20:50:00+00:00,16,187.937500,15.687500
4,2008-01-07 21:20:00+00:00,10,99.900000,2.000000
5,2008-01-13 01:30:00+00:00,2,1417.000000,49.500000
6,2008-02-12 20:20:00+00:00,3,1985.666667,124.666667
7,2008-04-05 18:20:00+00:00,4,46.000000,6.000000


## 8) For each tag, determine the number of videos

Note: `tags` is a string of multiple tags separated by `|`. We'll treat `[none]` as no tag and remove it from counting.

In [51]:
all_tags = youtube["tags"].dropna().str.split("|").explode() #splitting tags like fun|music|comedy" → ["fun", "music", "comedy"].
# value_counts..counts how many times each tag appears in all videos. (Example: "fun" appears 200 times, "music" 150 times, etc)


tag_counts = all_tags.value_counts().reset_index() #explode takes each list and creates a new row for every tag. (one video with ["fun", "music", "comedy"] → 3 rows: "fun", "music", "comedy")


tag_counts.columns = ["tag", "count"]  #Renames the columns so the first column is "tag" and the second is "count".

print("Top 10 tags:")
tag_counts.head(10)

Top 10 tags:


,tag,count
0,[none],35518
1,"""funny""",14834
2,"""comedy""",11900
3,"""2018""",10567
4,"""news""",5653
5,"""music""",5544
6,"""video""",5338
7,"""2017""",5334
8,"""humor""",4992
9,"""television""",4099


In [52]:
# Let us remove the none for more clarity: 

# Remove [none] and strip extra quotes
all_tags = youtube["tags"].dropna().str.split("|").explode()
all_tags = all_tags[all_tags != "[none]"]  # remove [none]
all_tags = all_tags.str.replace('"', '')   # remove extra quotes

tag_counts = all_tags.value_counts().reset_index()
tag_counts.columns = ["tag", "count"]

print("Top 10 tags:")
tag_counts.head(10)

Top 10 tags:


,tag,count
0,funny,14940
1,comedy,12289
2,2018,10903
3,news,5986
4,music,5862
5,video,5558
6,2017,5521
7,humor,5013
8,television,4109
9,review,4046


## 9) Find the tags with the largest number of videos

In [53]:
print("Top 10 tags:")
tag_counts.head(10)


Top 10 tags:


,tag,count
0,funny,14940
1,comedy,12289
2,2018,10903
3,news,5986
4,music,5862
5,video,5558
6,2017,5521
7,humor,5013
8,television,4109
9,review,4046


## 10) For each (tag, country) pair, compute average ratio likes/dislikes

We use `like_ratio` computed earlier. We compute mean per (tag, country) pair.

In [54]:
youtube["tags_split"] = youtube["tags"].str.split("|")
tags_country = youtube.explode("tags_split")
tag_country_ratio = tags_country.groupby(["tags_split", "country"])["like_ratio"].mean().reset_index()
tag_country_ratio.head()


,tags_split,country,like_ratio
0,,CA,8.618739
1,,DE,24.344225
2,,FR,16.424636
3,,IN,7.652585
4,,JP,9.894877


## 11) For each (trending_date, country) pair, the video with the largest number of views

In [55]:
# ensure trending_date_parsed exists (we parsed earlier)
# for grouping by original trending_date string we can use trending_date_parsed or trending_date

# find index of max views per (trending_date, country) on the time-series youtube
idx = youtube.groupby(['trending_date','country'])['views'].idxmax()  # index of row with max views in each group
top_video_per_day = youtube.loc[idx].reset_index(drop=True)  # select those rows

top_video_per_day[['trending_date','country','video_id','title','views']].head(15)


,trending_date,country,video_id,title,views
0,17.01.12,CA,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,56367282
1,17.01.12,DE,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,56367282
2,17.01.12,FR,3VbHg5fqBYw,Avengers: Infinity War Trailer Tease,7281189
3,17.01.12,GB,TyHvyGVs42U,"Luis Fonsi, Demi Lovato - Échame La Culpa",143408235
4,17.01.12,IN,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,56367282
5,17.01.12,KR,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,56370607
6,17.01.12,MX,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,56370607
7,17.01.12,RU,3VbHg5fqBYw,Avengers: Infinity War Trailer Tease,7281189
8,17.01.12,US,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,56367282
9,17.02.12,CA,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,66637636


## 12) Divide `trending_date` into three columns: year, month, day

In [57]:
# trending_date format is 'yy.dd.mm' (string). We'll split into three columns
youtube['trending_date'] = youtube['trending_date'].astype(str)  # ensure string type
youtube[['td_year','td_day','td_month']] = youtube['trending_date'].str.split('.', expand=True)  # split into 3 parts

# convert to numeric and sensible names
youtube['td_year'] = '20' + youtube['td_year'].astype(str)  # make full year e.g. '17' -> '2017'
youtube[['td_year','td_month','td_day']].head(10)


,td_year,td_month,td_day
0,2017,11,14
1,2017,11,14
2,2017,11,14
3,2017,11,14
4,2017,11,14
5,2017,11,14
6,2017,11,14
7,2017,11,14
8,2017,11,14
9,2017,11,14


In [58]:
#something is weird, lets check for the unique dates: 
youtube['trending_date'].nunique()


205

In [59]:
# maybe the first 10 is happening to be the same, lets check for the diff values: 

youtube[['td_year','td_month','td_day']].drop_duplicates().sort_values(['td_year','td_month','td_day']).head(10)


,td_year,td_month,td_day
0,2017,11,14
196,2017,11,15
392,2017,11,16
586,2017,11,17
784,2017,11,18
982,2017,11,19
1178,2017,11,20
1373,2017,11,21
1569,2017,11,22
1768,2017,11,23


In [60]:
#maybe ill check for unique months and years too: 

youtube[['td_year','td_month']].drop_duplicates().sort_values(['td_year','td_month'])


,td_year,td_month
0,2017,11
3341,2017,12
9422,2018,01
15080,2018,02
20500,2018,03
26572,2018,04
31253,2018,05
37296,2018,06


## 13) For each (month, country) pair, the video with the largest number of views

In [61]:
# use td_month column we created and country to group
idx_month = youtube.groupby(['td_month','country'])['views'].idxmax()  # index of max views per month-country pair
top_video_per_month = youtube.loc[idx_month].reset_index(drop=True)

top_video_per_month[['td_month','country','video_id','title','views']].head(15)


,td_month,country,video_id,title,views
0,01,CA,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,43067983
1,01,DE,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
2,01,FR,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
3,01,GB,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,90598955
4,01,IN,dfnCAmr569k,"Taylor Swift - End Game ft. Ed Sheeran, Future",42019590
5,01,KR,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
6,01,MX,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,31680160
7,01,RU,dfnCAmr569k,"Taylor Swift - End Game ft. Ed Sheeran, Future",23198594
8,01,US,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,57951412
9,02,CA,xpVfcZ0ZcFM,Drake - God’s Plan,47362934


## 14) Read all JSON files with the video categories (one JSON per country)

We'll build a DataFrame `categories_df` with columns: country, id, title

In [63]:
# find category JSON files
json_files = glob.glob('data/json/*category_id.json')

categories = []  # list to store mappings

for file in json_files:
    country = file.split('/')[-1][:2]  # extract country code from filename
    with open(file, 'r', encoding='utf-8') as f:  # no extra space
        data = json.load(f)
    items = data.get('items', [])
    for item in items:
        categories.append({'country': country, 'id': str(item['id']), 'title': item['snippet']['title']})

categories_df = pd.DataFrame(categories)  # create dataframe of categories
categories_df.head(12)



,country,id,title
0,js,1,Film & Animation
1,js,2,Autos & Vehicles
2,js,10,Music
3,js,15,Pets & Animals
4,js,17,Sports
5,js,18,Short Movies
6,js,19,Travel & Events
7,js,20,Gaming
8,js,21,Videoblogging
9,js,22,People & Blogs


In [65]:
# We are getting the same country code..That usually happens because of the path separator issue on Windows.

json_files = glob.glob('data/json/*category_id.json')
categories = []

for file in json_files:
    country = os.path.basename(file)[:2]  # reliable on Windows & Linux
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    items = data.get('items', [])
    for item in items:
        categories.append({
            'country': country,
            'id': str(item['id']),
            'title': item['snippet']['title']
        })

categories_df = pd.DataFrame(categories)
print(categories_df['country'].value_counts())

country
US    32
CA    31
DE    31
FR    31
IN    31
GB    31
JP    31
KR    31
MX    31
RU    31
Name: count, dtype: int64


## 15) For each country, determine how many videos have a category that is not assignable (i.e., category id not present in the json file for that country)

In [67]:
# ensure category_id is string for safe merge
youtube['category_id'] = youtube['category_id'].astype(str)
categories_df['id'] = categories_df['id'].astype(str)

# merge youtube with categories on (category_id, country) -> left merge so all youtube rows are kept
merged = youtube.merge(categories_df, left_on=['category_id','country'], right_on=['id','country'], how='left', indicator=True)

# rows where the merge did not find a category will have _merge == 'left_only'
unassignable = merged[merged['_merge'] == 'left_only']

unassigned_count = unassignable.groupby('country')['video_id'].nunique().reset_index(name='unassignable_count')

unassigned_count


,country,unassignable_count
0,CA,47
1,DE,180
2,FR,78
3,GB,6
4,IN,28
5,JP,13
6,KR,114
7,MX,125
8,RU,1089


## ✅ Save final datasets and wrap up

In [ ]:
# save cleaned full youtube dataframe and exploded tags for later use
youtube.to_csv('final_youtube_full_steps1-15.csv', index=False, encoding='utf-8')
tags_exploded.to_csv('final_youtube_tags_exploded_steps1-15.csv', index=False, encoding='utf-8')

print('Saved: final_youtube_full_steps1-15.csv')
print('Saved: final_youtube_tags_exploded_steps1-15.csv')
